Run this after `condition_measures.ipynb` and `condition_contrasts.ipynb`: their outputs are some of this notebook's inputs. 

In [1]:
import numpy as np
import pandas as pd

import findlay2025a as f25a
from findlay2025a.constants import Experiments as Exps
from findlay2025a.constants import Files

Inspection of CNPIX14-Francis, COW, imec0, M1 cortical iDelta vs. time shows a clear, slow, positive drift in the median ieta value over 48h. The expected homeostatic changes are visible on top of this, and could probably be recovered by detrending, but it would be better to solve the problem at its root. 

Inspection of the actual LFP does not reveal why this slow drift in ieta would appear. It isn't like channels go good/bad, or LFP amplitude changes in general. 

In [2]:
measure_exclusions = [
    (
        ("CNPIX14-Francis", Exps.COW, "imec0"),
        tuple(f25a.core.CONDITION_DISPLAY_NAMES),
    ),  # Excludes all conditions
]

In [3]:
regions = {
    "M1": "MOT",
    "M2": "MOT",
    "V1": "VIS",
    "V2": "VIS",
    "PPC": "PPC",
    "mPPC": "PPC",
    "Cg1": "FRC",
    "MO": "FRC",
    "VO": "FRC",
    "PrL": "FRC",
    "IL": "FRC",
    "EC": "PHR",
    "Cx": "PHR",
}

measure_display_names = {
    "cx_mean_zlog_idelta": "Cortical.iDelta",
    "cx_mean_zlog_ieta": "Cortical.iEta",
}


def apply_measure_exclusions(
    df: pd.DataFrame,
    exclusions: list[tuple] = measure_exclusions,
    multiprobe: bool = True,
):
    essential_cols = ["condition", "subject", "experiment"]
    if multiprobe:
        essential_cols = essential_cols + ["probe"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    for criteria, conditions in exclusions:
        loc = (df[essential_cols[1:]] == criteria).all(axis=1)
        drop = loc & df["condition"].isin(conditions)
        df = df[~drop]

    return df.reset_index(drop=True)


def use_measure_display_names(
    df: pd.DataFrame,
    measure_names: dict[str, str] = measure_display_names,
    minimize: bool = True,
    multiprobe: bool = True,
) -> pd.DataFrame:
    essential_cols = ["condition", "subject", "experiment"]
    if multiprobe:
        essential_cols = essential_cols + ["acronym", "spw_probe"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    if minimize:
        df = df.loc[:, essential_cols + [x for x in measure_names if x in df.columns]]

    df.rename(columns=measure_names, inplace=True)
    return df


def use_condition_display_names(
    df: pd.DataFrame,
    condition_names: dict[str, str] = f25a.core.CONDITION_DISPLAY_NAMES,
    minimize: bool = True,
    multiprobe: bool = True,
) -> pd.DataFrame:
    essential_cols = ["condition", "subject", "experiment"]
    if multiprobe:
        essential_cols = essential_cols + ["acronym", "spw_probe"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    if minimize:
        df = df[df["condition"].isin(condition_names)]

    df["condition"] = df["condition"].map(f25a.core.CONDITION_DISPLAY_NAMES)
    return df


def use_experiment_display_names(
    df: pd.DataFrame, names: dict[str, str] = f25a.core.EXPERIMENT_DISPLAY_NAMES
) -> pd.DataFrame:
    df["experiment"] = df["experiment"].map(names)
    return df


def _is_spw_probe(row: pd.Series) -> bool:
    return f25a.core.get_spw_probe(row["experiment"], row["subject"]) == row["probe"]


def _get_essential_measures(band: str) -> pd.DataFrame:
    nbsh = f25a.core.get_project("seahorse")
    file_ = nbsh.get_project_file(f"cx_{band}_condition_means.pqt")

    df = pd.read_parquet(file_).reset_index()
    df = apply_measure_exclusions(df)
    df["spw_probe"] = df.apply(_is_spw_probe, axis=1)
    df = use_measure_display_names(df)
    df = use_condition_display_names(df)
    df = use_experiment_display_names(df)
    return df


def copy_from_spw_probe(
    mdf: pd.DataFrame, on: list[str], spw_probe_cols: list[str]
) -> pd.DataFrame:
    bdf = mdf.reset_index(drop=True)
    for i in range(len(bdf)):
        sel = bdf.loc[i, on]
        spw_probe = bdf.loc[i, "spw_probe"]
        if not spw_probe:
            copy_from = bdf.loc[
                (bdf[on] == sel).all(axis=1) & (bdf["spw_probe"]),
                spw_probe_cols,
            ]
            assert len(copy_from) == 1
            bdf.loc[i, spw_probe_cols] = copy_from.squeeze()
    return bdf


def apply_spw_probe_exclusions(
    df: pd.DataFrame, cm: pd.DataFrame, on: list[str]
) -> pd.DataFrame:
    def is_included(row: pd.Series) -> bool:
        return any((cm[on] == row[on]).all(axis=1))

    df = df[df.apply(is_included, axis=1)]
    return df


def get_essential_measures() -> pd.DataFrame:
    nbsh = f25a.core.get_project("seahorse")
    cm_file = nbsh.get_project_file(Files.EXPORTED_CONDITION_MEASURES)

    cm = pd.read_parquet(cm_file)
    spw_probe_cols = cm.columns
    cm["spw_probe"] = True

    df_delta = _get_essential_measures("idelta")
    df_eta = _get_essential_measures("ieta")
    df = pd.merge(
        df_delta,
        df_eta,
        on=["condition", "subject", "experiment", "acronym", "spw_probe"],
        how="outer",
    )
    df = apply_spw_probe_exclusions(df, cm, ["subject", "experiment", "condition"])
    mdf = pd.merge(
        df, cm, on=["condition", "subject", "experiment", "spw_probe"], how="outer"
    )
    mdf = copy_from_spw_probe(
        mdf, ["subject", "experiment", "condition"], spw_probe_cols
    )
    mdf["region"] = mdf["acronym"].map(regions).astype("category")
    return mdf


In [4]:
contrast_exclusions = [
    (
        ("CNPIX14-Francis", Exps.COW, "imec0"),
        ("rebound", "surge", "decline", "incline"),
    ),  # Excludes all contrasts
]

contrast_display_names = {
    "cx_mean_zlog_idelta_nrem_rebound": "Cortical.iDelta.Rebound",
    "cx_mean_zlog_idelta_nrem_rec_decline": "Cortical.iDelta.REC.Decline",
    "cx_mean_zlog_idelta_nrem_surge": "Cortical.iDelta.Surge",
    "cx_mean_zlog_idelta_nrem_bsl_decline": "Cortical.iDelta.BSL.Decline",
}


def apply_contrast_exclusions(
    df: pd.DataFrame,
    exclusions: list[tuple] = contrast_exclusions,
    multiprobe: bool = True,
) -> pd.DataFrame:
    essential_cols = ["subject", "experiment"]
    if multiprobe:
        essential_cols = essential_cols + ["probe"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    for row_loc, col_loc in exclusions:
        cols = [
            col for col in df.columns if any(substr in col for substr in col_loc)
        ]  # Columns whose values to drop
        rows = (df[essential_cols] == row_loc).all(axis=1)
        df.loc[rows, cols] = np.nan

    return df


def use_contrast_display_names(
    df: pd.DataFrame,
    names: dict[str, str] = contrast_display_names,
    minimize: bool = True,
    multiprobe: bool = True,
) -> pd.DataFrame:
    essential_cols = ["subject", "experiment"]
    if multiprobe:
        essential_cols = essential_cols + ["acronym", "spw_probe"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    if minimize:
        df = df.loc[:, essential_cols + [x for x in names if x in df.columns]]

    df.rename(columns=names, inplace=True)
    return df


def _get_essential_contrasts(band: str) -> pd.DataFrame:
    nbsh = f25a.core.get_project("seahorse")
    file_ = nbsh.get_project_file(f"cx_{band}_condition_contrasts.pqt")
    # Exclusions are already applied in the file

    df = pd.read_parquet(file_).reset_index()
    df = apply_contrast_exclusions(df)
    df["spw_probe"] = df.apply(_is_spw_probe, axis=1)
    df = use_contrast_display_names(df)
    df = use_experiment_display_names(df)
    return df


def get_essential_contrasts() -> pd.DataFrame:
    nbsh = f25a.core.get_project("seahorse")
    cc_file = nbsh.get_project_file("condition_contrasts.pqt")

    cc = pd.read_parquet(cc_file)
    spw_probe_cols = cc.columns
    cc["spw_probe"] = True

    df_delta = _get_essential_contrasts("idelta")
    if False:
        df_eta = _get_essential_contrasts("ieta")
        df = pd.merge(
            df_delta,
            df_eta,
            on=["subject", "experiment", "acronym", "spw_probe"],
            how="outer",
        )  # Not used for this project, doesn't apply to these contrasts anyways.
    else:
        df = df_delta
    mdf = pd.merge(df, cc, on=["subject", "experiment", "spw_probe"], how="outer")
    mdf = copy_from_spw_probe(mdf, ["subject", "experiment"], spw_probe_cols)
    mdf["acronym"] = mdf["acronym"].cat.remove_unused_categories()
    mdf["region"] = mdf["acronym"].map(regions).astype("category")
    return mdf

In [5]:
measures = get_essential_measures()
contrasts = get_essential_contrasts()

In [6]:
nbsh = f25a.core.get_project("seahorse")
measures.to_parquet(nbsh.get_project_file("multicortical_condition_measures.pqt"))
contrasts.to_parquet(nbsh.get_project_file("multicortical_condition_contrasts.pqt"))